In [1]:
import torch.nn as nn
import torchvision
import torch.optim
from models import MyVisionTransformer

In [2]:
# Dataset loading
folder_train = torchvision.datasets.ImageFolder(
    root="../train_images",
    transform=torchvision.transforms.Compose([
        torchvision.transforms.Resize((224, 224)),
        torchvision.transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
        torchvision.transforms.RandomGrayscale(0.25),
        torchvision.transforms.ToTensor(),
    ])
)

train_loader = torch.utils.data.DataLoader(
    folder_train,
    batch_size=32,
    shuffle=True
)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MyVisionTransformer().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.003)
model.train()

C:\Users\Programming\PJPProject\BIA\.venv\lib\site-packages\torch\nn\modules\transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


MyVisionTransformer(
  (patch_layer): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (linear1): Linear(in_features=768, out_features=3072, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=3072, out_features=768, bias=True)
        (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (classification_head): Sequential(
    (0): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (1): Linear(in_features=768, out_features=1, bias=True)
  )
)

In [4]:
# Train
num_episodes = 20

best_loss = float("inf")
for episode in range(num_episodes):
    episode_loss = 0

    for images, labels in train_loader:
        # Preprocess
        images, labels = images.to(device), labels.float().to(device).view(-1, 1)

        # Reset gradients
        optimizer.zero_grad()

        # Feed stuff into model
        outputs = model(images)

        # Compute loss and backpropagate
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Accumulate loss
        episode_loss += loss.item()

    print(f"Episode {episode}, loss: {episode_loss}")

    if episode_loss < best_loss:
        best_loss = episode_loss
        torch.save(model.state_dict(), "../models/parking_vit.pth")
        print("\tSaving!")


Episode 0, loss: 90.18940916657448
	Saving!
Episode 1, loss: 76.59919267892838
	Saving!
Episode 2, loss: 67.10758204758167
	Saving!
Episode 3, loss: 54.75754349672934
	Saving!
Episode 4, loss: 14.683990838937461
	Saving!
Episode 5, loss: 6.550026143086143
	Saving!
Episode 6, loss: 6.308090866135899
	Saving!
Episode 7, loss: 6.365665448887739
Episode 8, loss: 7.172850830946118
Episode 9, loss: 7.295977511093952
Episode 10, loss: 4.893221385427751
	Saving!
Episode 11, loss: 6.236641188152134
Episode 12, loss: 4.243655933620175
	Saving!
Episode 13, loss: 5.183591270353645
Episode 14, loss: 4.541741847642697
Episode 15, loss: 6.642425086960429
Episode 16, loss: 5.187232234922703
Episode 17, loss: 5.011467203410575
Episode 18, loss: 3.505121786009113
	Saving!
Episode 19, loss: 3.0266292620799504
	Saving!
